# Setup

In [24]:
import duckdb
from splink import DuckDBAPI, Linker, SettingsCreator, block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
    count_comparisons_from_blocking_rule,
    n_largest_blocks,
)

In [25]:
con = duckdb.connect()

df_l = con.read_parquet("./perturbed_data_l.parquet")
df_r = con.read_parquet("./perturbed_data_r.parquet")

db_api = DuckDBAPI(connection=con)

# Basic blocking

In [26]:
brs = [
    block_on("fname", "substring(sname, 1, 2)"),
    block_on("dob"),
    block_on("address", arrays_to_explode=["address"]),
    block_on("substr(fname, 1, 2)", "year(dob)"),
    block_on("year(dob)", "sname"),
]

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[df_l, df_r],
    blocking_rules=brs,
    link_type='link_only',
    db_api=db_api,
)

alt.Chart(...)

In [27]:
n_largest_blocks(
    table_or_tables=[df_l, df_r],
    blocking_rule=block_on("fname"),
    link_type='link_only',
    db_api=db_api,
    n_largest=10,
).as_pandas_dataframe()

,key_0,count_l,count_r,block_count
0,HSTG,1605,1631,2617755
1,SBYWSH,1577,1653,2606781
2,VTNAET,1586,1562,2477332
3,CUQDU,1557,1554,2419578
4,OUOYNT,1551,1520,2357520
5,RGQRXJ,1483,1492,2212636
6,VONZZK,1422,1424,2024928
7,XLSAUBK,1430,1403,2006290
8,XZUGDGW,1350,1404,1895400
9,KOSEA,1378,1371,1889238


# Advanced blocking - maximum block size

Sometimes a field is very skewed - a few values have a very high frequency, while most values have a relatively low frequency. Names are often like this. It may be useful to block on these fields, but only using the values with a low frequency. 

If you do this, some records will be excluded because their value for that field is too common. This means that you need to make sure there are other blocking rules that can pick up potential links for those records. The following example shows this by blocking on first names that appear less than 500 times, and following that by blocking on first name and surname (with no restriction on frequency). 

In [28]:
df_concat = con.sql(
    """
    select *, 'df_l' as source_dataset
    from df_l 
    
    union all
    
    select *, 'df_r' as source_dataset
    from df_r
    """
)

# add a 
fn_counts = con.sql(
    """
    select
        fname,
        case
            when count(*) < 500 then fname
            else NULL
        end as fn_rare
    from df_concat
    group by fname
    """
)

df = con.sql(
    """
    select *
    from df_concat
    left join fn_counts
    using (fname)
    """
)

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[df],
    blocking_rules=[block_on("fn_rare"), block_on("fname", "sname")],
    link_type='link_only',
    db_api=db_api,
)

alt.Chart(...)

This approach limits the number of comparisons for fname to 1,600,000, while there are an additional 300,000 record pairs that share both first name and surname, but were excluded from the first blocking rule. 

These two rules combined are much lower than just blocking on fname, which generates about 67,000,000 comparisons.


# Assessing blocking rules

Above, we have focused on counting how many comparisons the blocking rules generate, but the other major consideration is how many true matches are captured by the blocking rules. We have no way of knowing this for sure, because we don't know what the true matches are. 

One way to roughly approximate the coverage of a blocking rule is to look at the size of the blocks for each record. Some of these blocks will have size 1 - so that record won't be compared with anything else. You can extend this by looking at the maximum block size for that record across all of the blocking rules. This means that we can roughly see the overall coverage for a set of blocking rules.

In [29]:
counts_1 = con.sql(
    """
    select
        fname,
        sname,
        count(*) as n
    from df_concat
    where fname is not null and sname is not null
    group by fname, sname
    """
)

df_with_counts_1 = con.sql(
    """
    select 
        df_concat.unique_id,
        coalesce(n, 0) as n
    from df_concat
    left join counts_1
    using (fname, sname)
    """
)

counts_2 = con.sql(
    """
    select
        dob,
        count(*) as n
    from df_concat
    where dob is not null
    group by dob
    """
)

df_with_counts_2 = con.sql(
    """
    select
        df_concat.unique_id,
        coalesce(n, 0) as n
    from df_concat
    left join counts_2
    using (dob)
    """
)

max_block_sizes = con.sql(
    """
    select
        unique_id,
        max(n) as n
    from (
        select * 
        from df_with_counts_1

        union all

        select *
        from df_with_counts_2
    )
    group by unique_id
    """
)

con.sql(
    """
    select count(*)
    from max_block_sizes
    where n <= 1
    """
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            3 │
└──────────────┘

# Advanced blocking - Locality Sensitive Hashing

Locality Sensitive Hashing (LSH) is a technique for randomly placing records into blocks so that records that are more similar are more likely to be placed in the same block. This is a more advanced technique to implement, but it is a more general approach to blocking than selecting blocking rules by hand.

The idea behind LSH is to split each record into a collection of q-grams (overlapping strings of length q). The similarity of two records is measured using the Jaccard similarity of the two sets of q-grams, which is the size of their intersection divided by the size of their union. For each record, some number of qgrams are selected using min-hashes (see resources for more information). These are divided up into b bands of r qgrams. Each record is placed into b blocks, and each block of those blocks is the set of records that agree on those r qgrams. 
The numbers b and r are the two main parameters for LSH. Increasing b means that records have a higher chance to be placed in the same block, and increasing r means that two records have to be more similar to be placed into the same block. 

LSH has some of the same problems as selecting blocking rules by hand. You can still get some high frequency values that create very large blocks, which generates a lot of comparisons. LSH can be combined with techniques that filter out high-frequency blocks (or rank based filtering as used in the first resource).

LSH is not implemented in Splink.

**Resources**

- [Locality Sensitive Hashing with Temporal and Spatial Constraints for
Efficient Population Record Linkage](https://dl.acm.org/doi/pdf/10.1145/3511808.3557631)
- [UNIQUE ENTITY ESTIMATION WITH APPLICATION TO THE SYRIAN CONFLICT](https://www.jstor.org/stable/26542562)
- [Blocking and Filtering Techniques for Entity Resolution:
A Survey](https://dl.acm.org/doi/pdf/10.1145/3377455)



In [48]:
# very rough implementation to demonstrate LSH
def create_q_grams(connection, tbl, blocking_cols, id_column, q):
    select_blocking_cols = ", ".join(blocking_cols)
    
    tbl_with_blocking_string = connection.sql(
        f"""
        select 
            {id_column},
            concat_ws('_', {select_blocking_cols}) as blocking_string
        from tbl
        """
    )

    tbl_with_qgrams = connection.sql(
        f"""
        select
            {id_column},
            list_transform(
                range(1, length(blocking_string) - {q} + 2),
                lambda i : blocking_string[i : i+{q}-1]
            ) as qgrams
        from tbl_with_blocking_string
        """
    )
    
    return tbl_with_qgrams


def create_lsh_blocks(connection, tbl, blocking_cols, id_column, q, b, r):
    qgrams = create_q_grams(
        connection=connection,
        tbl=tbl,
        blocking_cols=blocking_cols,
        id_column=id_column,
        q=q,
    )

    min_hash_exprs = [
        f"list_min(list_transform(qgrams, lambda x : hash(x, {i}))) as min_hash_{i}"
        for i in range(b * r)
    ]

    select_hashes = ", ".join(min_hash_exprs)

    min_hashes = connection.sql(
        f"""
        select
            {id_column},
            qgrams,
            {select_hashes}
        from qgrams
        """
    )
    # block_id_exprs = []
    # for b in range(0, b):
    #     hashes = ", ".join([f"min_hash_{i}" for i in range(b*r, b*r+r)])
    #     block_id_expr = f"hash({hashes}) as block_id_{b}"
    #     block_id_exprs.append(block_id_expr)

    # blocks = connection.sql(
    #     f"""
    #     select
    #         {id_column},
    #         {", ".join(block_id_exprs)}
    #     from min_hashes
    #     """
    # )
    
    return min_hashes

blocks = create_lsh_blocks(con, df, ["fname", "sname", "strftime(dob, '%Y%m%d')"], "unique_id", q=3, b=4, r=4)

In [49]:
create_q_grams(con, df, ["fname", "sname", "sex", "strftime(dob, '%Y%m%d')"], "unique_id", q=3)

┌───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ unique_id │                                                               qgrams                                                               │
│   int64   │                                                             varchar[]                                                              │
├───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│         0 │ [ZRB, RBA, BA_, A_B, _BR, BRJ, RJT, JTJ, TJZ, JZC, ZCQ, CQE, QE_, E_2, _2_, 2_2, _20, 200, 006, 061, 611, 110, 103]                │
│         1 │ [AEP, EPM, PMP, MPP, PP_, P_C, _CM, CMU, MUO, UOG, OGR, GRD, RDQ, DQS, QSK, SK_, K_2, _2_, 2_2, _20, 200, 004, 040, 405, 052, 520] │
│         2 │ [ADC, DCH, CHR, HRJ, RJ_, J_I, _IH, IHZ, HZV, ZVY, VYS, YSU, SUQ, UQ_, Q_1, _1_, 1_1, _19, 194, 945, 450

In [54]:
blocks = create_lsh_blocks(con, df, ["fname", "sname", "sex", "strftime(dob, '%Y%m%d')"], "unique_id", q=3, b=2, r=2)

In [55]:
blocks

┌───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ unique_id │                                                               qgrams                                                               │     min_hash_0      │     min_hash_1      │     min_hash_2      │     min_hash_3      │
│   int64   │                                                             varchar[]                                                              │       uint64        │       uint64        │       uint64        │       uint64        │
├───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│         0 │ [ZRB, RBA, BA_, A_B, _BR, BRJ, RJT, JTJ, TJZ, 

In [46]:
con.sql("select count(*) from df")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       185400 │
└──────────────┘

In [47]:
con.sql("select count(*) from blocks")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       185400 │
└──────────────┘

In [23]:
con.sql(
    """
    with t as (
        select 
            block_id_1,
            count(*) as n
        from blocks
        group by block_id_1
    )
    select n, count(*)
    from t 
    group by n
    order by n desc
    """
)

┌───────┬──────────────┐
│   n   │ count_star() │
│ int64 │    int64     │
├───────┼──────────────┤
│  1473 │            1 │
│  1025 │            1 │
│   609 │            1 │
│   589 │            1 │
│   543 │            1 │
│   538 │            1 │
│   521 │            1 │
│   494 │            1 │
│   482 │            1 │
│   476 │            1 │
│     · │            · │
│     · │            · │
│     · │            · │
│    10 │          309 │
│     9 │          461 │
│     8 │          576 │
│     7 │          781 │
│     6 │         1096 │
│     5 │         1674 │
│     4 │         2599 │
│     3 │         4798 │
│     2 │        16932 │
│     1 │        40722 │
├───────┴──────────────┤
│ 145 rows   2 columns │
│ (20 shown)           │
└──────────────────────┘